<a href="https://colab.research.google.com/github/karimekirey/redesneuronales/blob/main/Confiabilidad_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto Final de Redes Neuronales Artificiales
"Confiabilidad de Productos en Amazon Basado en reseñas"

Equipo 7:
Karime Kirey Sánchez Andrade 1914174
Lázaro Gael Cortez Moncada



---



PREPROCESAMIENTO DE LOS DATOS
---



In [2]:
import pandas as pd
import numpy as np
from collections import Counter

# el dataset puede obtenerse del siguiente link https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
# Importamos el dataset utilizado contiene reseñas de artículos de la categoría de Oficina. Especificamos que los encabezados están en la fila 0

dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')
df=pd.DataFrame(dtf) #Convertimos el dataset en un Dataframe con pandas
print(df.head())

df.shape
df.info()

<ipython-input-2-68fea294cbff>:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')


  marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N  

In [3]:
df['review_date'] = pd.to_datetime(df['review_date'])
reviews_2015 = df[df['review_date'].dt.year == 2015]
count_reviews_2015 = len(reviews_2015)
print("El número de reseñas de 2015 es: ", count_reviews_2015)


El número de reseñas de 2015 es:  815817


In [4]:
#En esta parte del código se borraron las columnas que no se utilizarán con la función de pandas "drop"
#labeling the columns
columnas_quitar = ['marketplace', 'customer_id', 'review_id', 'product_parent', 'vine']
datos=df.drop(columnas_quitar, axis=1)
datos = datos[datos['review_date'].dt.year == 2015]
print(datos.head(20))
datos.shape

    product_id                                      product_title  \
0   B001BM2MAC     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet   
1   B00DZYEXPQ          Dust-Off Compressed Gas Duster, Pack of 4   
2   B00RTMUHDW  Amram Tagger Standard Tag Attaching Tagging Gu...   
3   B00D7H8XB6  AmazonBasics 12-Sheet High-Security Micro-Cut ...   
4   B001XCWP34  Derwent Colored Pencils, Inktense Ink Pencils,...   
5   B004J2NBCO  Quartet Magnetic Dry-Erase Weekly Organizer, 6...   
6   B00MOPAG8K  KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...   
7   B003AHIK7U              Bible Dry Highlighting Kit (Set of 4)   
8   B006TKH2RO  Parker Ingenuity Large Black Rubber & Metal CT...   
9   B00W61M9K0                                RFID Card Protector   
10  B00BXOGI3A  The Original Stand Steady Standing Desk - Inst...   
11  B0018RHWGE  Zebra Sparky-1 WKP1-10C Fluorescent Marker (10...   
12  B00IA8H2OG  E-Z Ink (TM) Compatible Ink Cartridge Replacem...   
13  B007TOIA66                   E

(815817, 10)

In [5]:
#limpieza de palabras como "is", "the"

import re
import string
from gensim.parsing.preprocessing import remove_stopwords

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = remove_stopwords(text)
    return text

In [6]:
datos['review_body'] = datos['review_body'].fillna('').astype(str) #algunas reseñas contienen valores numéricos. Se convierte todo a str para 
datos['review_body'] = datos['review_body'].apply(clean_text)

In [7]:
print(df['verified_purchase'].dtype)

object


In [8]:
mapping = {'Y': 1, 'N': 0}
datos['verified_purchase'] = datos['verified_purchase'].map(mapping)

In [9]:
#dataframe con reseñas solamente de 2015
datos = datos.iloc[1:, :]
from IPython.display import display
display(datos.head(10))


,product_id,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
1,B00DZYEXPQ,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,1,"Phffffffft, Phfffffft. Lots of air, and it's C...",whats commodity item come worldbr having need ...,2015-08-31
2,B00RTMUHDW,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,1,but I am sure I will like it.,havent sure like,2015-08-31
3,B00D7H8XB6,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,1,and the shredder was dirty and the bin was par...,labeled received clearly box previously opened...,2015-08-31
4,B001XCWP34,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,1,Four Stars,gorgeous colors easy use,2015-08-31
5,B004J2NBCO,"Quartet Magnetic Dry-Erase Weekly Organizer, 6...",Office Products,5,0.0,0.0,1,Five Stars,perfect planning weekly meals removrd want con...,2015-08-31
6,B00MOPAG8K,KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...,Office Products,5,0.0,0.0,0,Five Stars,gold plated fusers best fail,2015-08-31
7,B003AHIK7U,Bible Dry Highlighting Kit (Set of 4),Office Products,5,6.0,6.0,1,Won't bleed through pages!,highlighters bible year love highlight alot bi...,2015-08-31
8,B006TKH2RO,Parker Ingenuity Large Black Rubber & Metal CT...,Office Products,5,0.0,0.0,1,Nice pen!,heavy pen writes ive couple weeks received com...,2015-08-31
9,B00W61M9K0,RFID Card Protector,Office Products,5,0.0,0.0,1,Credit card sleeves,sure work sent quickly fits wallet slots,2015-08-31
10,B00BXOGI3A,The Original Stand Steady Standing Desk - Inst...,Office Products,4,0.0,0.0,0,Four Stars,great product price easy stable easily adjustable,2015-08-31


In [10]:
##FILTRAR PARA VER CUÁNTAS RESEÑAS TIENE UN PRODUCTO

filtered_data = datos.loc[df['product_title'] == 'Bible Dry Highlighting Kit (Set of 4)']

# Imprimir los datos filtrados
print(filtered_data)

        product_id                          product_title product_category  \
7       B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
11032   B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
23553   B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
51932   B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
67358   B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
79122   B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
113693  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
116064  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
198196  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
239080  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
251550  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Office Products   
254288  B003AHIK7U  Bible Dry Highlighting Kit (Set of 4)  Offic

# ETIQUETACIÓN DE LOS DATOS: POLARIZACIÓN DE RESEÑAS

In [12]:
#ETIQUETACIÓN
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Crea una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Define una oración para analizar
datos = pd.DataFrame(datos['review_body'])

def obtener_sentimiento(reseña):
    scores = sia.polarity_scores(reseña)
    sentimiento = 'Positivo' if scores['compound'] >= 0 else 'Negativo'
    positividad = scores['compound']
    return sentimiento, positividad

# Calificar el sentimiento de todas las reseñas y agregar las columnas correspondientes
datos[['sentimiento', 'positividad']] = datos['review_body'].apply(lambda x: pd.Series(obtener_sentimiento(x)))

# Imprimir el DataFrame con las columnas de sentimiento y positividad
print(datos)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                              review_body sentimiento  \
1       whats commodity item come worldbr having need ...    Positivo   
2                                        havent sure like    Negativo   
3       labeled received clearly box previously opened...    Positivo   
4                                gorgeous colors easy use    Positivo   
5       perfect planning weekly meals removrd want con...    Positivo   
...                                                   ...         ...   
815818  packaged mailing bag arrived packaging damaged...    Positivo   
815819                         cheap broke matter seconds    Negativo   
815820                                   wish sets useful    Positivo   
815821  satisfied purchase perfect replacement toners ...    Positivo   
815822  purchased christmas gift item shipped received...    Positivo   

        positividad  
1            0.0000  
2           -0.4717  
3            0.0258  
4            0.7845  
5            

In [15]:
display(datos.head(30))

,review_body,sentimiento,positividad
1,whats commodity item come worldbr having need ...,Positivo,0.0000
2,havent sure like,Negativo,-0.4717
3,labeled received clearly box previously opened...,Positivo,0.0258
4,gorgeous colors easy use,Positivo,0.7845
5,perfect planning weekly meals removrd want con...,Positivo,0.6124
6,gold plated fusers best fail,Positivo,0.1779
7,highlighters bible year love highlight alot bi...,Positivo,0.6377
8,heavy pen writes ive couple weeks received com...,Positivo,0.9118
9,sure work sent quickly fits wallet slots,Positivo,0.3182
10,great product price easy stable easily adjustable,Positivo,0.8910




---
DIVISIÓN SET DE ENTRENAMIENTO Y VALIDACIÓN
---



# NORMALIZACIÓN DE VALORES NUMÉRICOS


In [ ]:
#   solo se normalizará el set de entrenamiento
#   Al haber rangos diferentes en el rating y el número total de votos, se normalizarán a una escala de -1 a 1 para evitar que una característica predomine más que otras
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data['star_rating'] = scaler.fit_transform(data['star_rating'].values.reshape(-1, 1))
data['total_votes'] = scaler.fit_transform(data['total_votes'].values.reshape(-1, 1))
data['helpful_votes'] = scaler.fit_transform(data['helpful_votes'].values.reshape(-1, 1))